# 이상치 
- 결측치, 범위에서 벗어나는 데이터, 값이 크게 벗어나는 데이터 
- 분석 모델의 성능을 떨어뜨리거나 결과에 악영향을 미치는 경우가 발생 
- 이상치는 발견 시에 특정한 데이터로 대체, 삭제 
- 결측치를 확인하는 방법
    - isna()를 이용하여 결측치의 유무를 판단하고 sum()을 이용해서 결측치의 개수를 확인
    - info()를 이용하여 non-null count를 보고 결측치의 존재를 파악 
- 특정 범위에 벗어나는 데이터 확인하는 방법 
    - isin()을 이용하여 특정 데이터의 포함 여부를 확인하고 전제 데이터를 비트연산자(~) 부정을 하고 확인 가능 
    - value_counts()를 이용하여 데이터의 개수를 보고 확인 가능 
    - unique()를 이용하여 확인 가능 
- 극단치( 값이 크게 벗어나는 데이터 )
    - IQR 방식 
        - matplot 안에 boxplot의 이상치를 결정하는 방법 
        - Q3(3사분위수)와 Q1(1사분위수)를 이용하여 범위를 지정 
        - IQR = Q3 - Q1
        - 극단치의 경계 
            - 상단의 경계 : Q3 + 1.5 * IQR
            - 하단의 경계 : Q1 - 1.5 * IQR
        - 많은 데이터들이 극단치로 판단이 될 가능성이 존재
        - 1.5의 값을 조절해서 극단치의 범위를 변경

In [ ]:
# 라이브러리 로드 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine

In [ ]:
# 샘플 데이터를 로드 
wine_data = load_wine()

In [ ]:
wine_data

- sklearn 라이브러리에서 sample data의 키
    - data
        - 독립변수 
        - 머신러닝에서 문제 (학습 데이터)
    - target
        - 종속변수 
        - 독립변수를 이용하여 예측에 대한 답
    - frame
        - 독립변수와 종속변수 데이터를 데이터프레임으로 제공 
        - 데이터가 존재하는 경우와 존재하지 않는 경우가 있다. 
    - DESCR
        - 데이터의 정보를 확인 
    - feature_names
        - 피쳐(컬럼)의 이름을 의미
    - target_names
        - 종속변수의 의미 

In [ ]:
wine_data['DESCR']

In [ ]:
wine = pd.DataFrame(
    data = wine_data['data'], 
    columns = wine_data['feature_names']
)
wine.head()

In [ ]:
# target data로 데이터프레임에 추가 -> 파생변수 생성
wine['class'] = wine_data['target']

In [ ]:
wine.info()

In [ ]:
# 특정 컬럼을 선택해서 boxplot()를 그린다. 
plt.boxplot(wine['color_intensity'])
plt.show()

In [ ]:
# 사분위수를 describe()를 이용하여 출력이 가능
wine['color_intensity'].describe()['75%']

In [ ]:
# numpy를 이용하여 사분위수를 출력 
q_1, q_3 = np.percentile(
    wine['color_intensity'], 
    [25, 75]
)

In [ ]:
# IQR계산 : 3사분위수 - 1사분위수
iqr = q_3 - q_1
# 상단의 경계 값 : 3사분위수 + (1.5 * IQR) 
upper_whis = q_3 + (1.5 * iqr)
# 하단의 경계 값 : 1사분위수 - (1.5 * IQR)
lower_whis = q_1 - (1.5 * iqr)
print(upper_whis, lower_whis)

In [ ]:
# 상단의 경계 값보다 크거나 하단의 경계보다 작은 데이터를 확인 
flag = (wine['color_intensity'] > upper_whis) | (wine['color_intensity'] < lower_whis)

In [ ]:
wine.loc[flag, ]

- 극단치의 데이터를 제거하는 방법 
    - drop()를 이용하여 제거 
    - 극단치의 경계식인 flag를 부정(~)하여 필터링
    - 극단치의 데이터를 결측치로 대체하고 결측치를 제거하는 함수를 이용
        - 특정 컬럼들의 극단치를 확인하여 결측치로 대체하고 독립변수들의 결측치 존재 유무를 행 단위로 파악하여 극단치의 개수를 확인하고 제거할지 정한다. 
- 극단치의 데이터를 특정 데이터로 대체
    - Series = 단일데이터 -> 스리즈의 모든 value가 단일데이터로 변경 
        - series -> 극단치의 상단 / 하단 경계 , 특정 컬럼명  = 상단 / 하단 경계 값
    - 대입 연산자를 사용하기 전에 백업 데이터를 구성

In [ ]:
# 백업 데이터 생성
df = wine.copy()

In [ ]:
# 극단치의 경계의 데이터를 특정 값으로 대체 
# 상단의 경계보다 큰 조건식 -> 참인 경우 color_intensity 데이터를 upper_whis 대체
upper_flag = df['color_intensity'] > upper_whis
# 하단의 경계보다 작은 조건식 -> 참인 경우 color_intensity 데이터를 lower_whis 대체
lower_flag = df['color_intensity'] < lower_whis

In [ ]:
df.loc[upper_flag, 'color_intensity'] = upper_whis

In [ ]:
df.loc[lower_flag, 'color_intensity'] = lower_whis

In [ ]:
df.loc[flag, ]

- 극단치를 확인하고 처리할수 있는 함수 생성 
    - 매개변수 4개 
        - data -> 필수항목 -> 데이터프레임
        - cols -> 컬럼을 선택 -> 인자의 개수가 가변인 경우 -> 인자가 0개라면 모든 컬럼
        - cnt -> 극단치의 범위 지정 -> 기본값은 1.5
        - type -> 되돌려주는 데이터의 타입 -> 기본값은 'dict' -> dict인 경우에는 key를 컬럼의 이름 value는 해당 컬럼의 극단치에 해당하는 데이터프레임 -> df인 경우에는 극단치에 해당하는 데이터들을 결측치로 대체
    - 기준이 되는 컬럼의 개수만큼 반복 실행 
        - 컬럼의 데이터에서 iqr을 구하고 상단의 경계, 하단의 경계 값을 변수에 저장 
        - type이 dict이라면
            - 데이터프레임에서 해당 경계를 벗어나는 부분을 데이터프레임으로 생성 
            - 빈 dict에 데이터를 채워준다. 
            - 반복을 종료하고 dict를 되돌려준다
        - type이 df라면 
            - 데이터프레임에서 해당 경계를 벗어나느 부분을 결측치로 채워준다. 
            - 반복을 종료하고 데이터프레임을 되돌려준다


In [ ]:
def outlier_iqr(_data, *_cols, _cnt = 1.5, _type='dict'):
    # 입력받은 데이터프레임의 복사본을 생성 
    df = _data.copy()
    # _cols의 개수가 0개라면 모든 컬럼을 선택 
    if _cols:
        # _cols의 데이터가 존재하는 경우 
        cols = _cols
    else:
        # _cols에 데이터가 존재하지 않는 경우
        cols = df.columns
    # 빈 딕셔너리 생성 
    result = dict()
    # cols를 기준으로 반복문을 생성
    for col in cols:
        try:
            # 컬럼의 데이터가 숫자가 아니라면 에러가 발생 -> 예외처리 
            # 1사분위, 3사분위 수를 변수에 저장 
            q_1, q_3 = np.percentile( df[col] , [25, 75] )
            # iqr 생성
            iqr = q_3 - q_1
            # 상단 경계 값 생성 
            upper_whis = q_3 + (_cnt * iqr)
            # 하단 경계 값 생성
            lower_whis = q_1 - (_cnt * iqr)
            print(f"""
                    column : {col}, 
                    upper_whis : {upper_whis}, 
                    lower_whis : {lower_whis}
                """)
            # 조건식 생성 (상단에 경계보다 크거나 하단의 경계보다 작은 경우)
            flag = (df[col] > upper_whis) | (df[col] < lower_whis) 
            # _type에 따라 행동을 다르게 구현 
            if _type == 'dict':
                # 극단치의 경계 데이터를 필터링
                outlier = df.loc[flag, ]
                # 빈 딕셔너리에 outlier을 추가 
                result[col] = outlier
            elif _type == 'df': 
                # 극단치의 경계 밖에 있는 데이터를 결측치로 대체
                df.loc[flag, col] = np.nan
            else:
                print('_type의 값이 잘못되었습니다 dict, df를 선택하세요')
                return ''
        except Exception as e:
            print("error : ", e)
            print('column : ', col)
    if _type == 'df':
        result = df
    return result

In [ ]:
outlier = outlier_iqr(wine, _type = 'df')

In [32]:
outlier.isna().sum()

alcohol                         0
malic_acid                      3
ash                             3
alcalinity_of_ash               4
magnesium                       4
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 2
color_intensity                 4
hue                             1
od280/od315_of_diluted_wines    0
proline                         0
class                           0
dtype: int64

In [ ]:
# 결측치가 하나라도 존재하는 데이터를 출력하려면?
outlier.loc[
    outlier.isna().any(axis=1)
]

In [39]:
# 데이터에서 결측치가 2개 이상인 인덱스 데이터를 출력?
outlier_flag = outlier.isna().sum(axis=1) >= 2

outlier.loc[outlier_flag, ]

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
59,12.37,0.94,NaN,NaN,88.0,1.98,0.57,0.28,0.42,1.95,1.05,1.82,520.0,1.0
73,12.99,1.67,2.6,NaN,NaN,3.30,2.89,0.21,1.96,3.35,1.31,3.50,985.0,1.0
95,12.47,1.52,2.2,19.0,NaN,2.50,2.27,0.32,NaN,2.60,1.16,2.63,937.0,1.0
121,11.56,2.05,NaN,NaN,119.0,3.18,5.08,0.47,1.87,6.00,0.93,3.69,465.0,1.0


In [ ]:
# 결측치가 포함되어있는 데이터를 모두 제거 
outlier.dropna(axis=0)

In [43]:
outlier.loc[
    ~(outlier.isna().any(axis=1))
]

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,14.16,2.51,2.48,20.0,91.0,1.68,0.70,0.44,1.24,9.70,0.62,1.71,660.0,2.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2.0


# 범주형 데이터의 변환
- 범주형 변수는 학습 모델이 대부분 수학적인 연산으로 모델을 생성하는데 직접적으로 사용이 어렵기 때문에 특별 가공 처리 
- 더미 변수 사용 
- 더미변수는 범주형 데이터를 각각의 컬럼으로 생성 -> 0과 1로 데이터를 대입하여 포함 여부를 생성 

In [ ]:
wine_data['target_names'][2]

[np.str_('class_0'), np.str_('class_1'), np.str_('class_2')]

In [ ]:
# 범주형 변수를 생성 
wine['class'].map(
    lambda x : wine_data['target_names'][x]
)

In [56]:
wine['class'] = wine['class'].map(
    {
        0 : 'Class_0', 
        1 : 'Class_1', 
        2 : 'Class_2'
    }
)

In [57]:
wine['class'].value_counts()

class
Class_1    71
Class_0    59
Class_2    48
Name: count, dtype: int64

In [58]:
# 더비 변수를 생성하는 함수 
# 가져오다(get) + 더미들(dummies) = get_dummies()
pd.get_dummies(wine, columns=['class'])

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class_Class_0,class_Class_1,class_Class_2
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,True,False,False
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,True,False,False
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,True,False,False
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,True,False,False
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,False,False,True
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,False,False,True
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,False,False,True
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,False,False,True
